# Time dependent ITER simulation

In [ ]:
using Plots;
using FUSE
FUSE.ProgressMeter.ijulia_behavior(:clear);

In [ ]:
ini, act = FUSE.case_parameters(:ITER; init_from=:ods);

In [ ]:
using Interact
@manipulate for time0 in ini.time.pulse_shedule_time_basis
    ini0 = deepcopy(ini)
    ini0.time.simulation_start = time0
    plot(ini, color=:red)
    plot!(ini0, color=:blue)
end

In [ ]:
chk = FUSE.Checkpoint();

dd = IMAS.dd()
@time dd = FUSE.init(dd, ini, act);
chk[:init] = dd, ini, act;

In [ ]:
FUSE.digest(dd)

In [ ]:
using Interact
@manipulate for time0 in dd.pulse_schedule.flux_control.time
    plot(dd.pulse_schedule; time0)
end

In [ ]:
dd = chk[:init].dd

ini, _ = FUSE.case_parameters(:ITER; init_from=:scalars);
ini.time.simulation_start = 100.0

FUSE.init(dd, ini, act; initialize_hardware=false);

chk[:tinit] = (dd, ini, act);

In [ ]:
FUSE.digest(dd)

In [ ]:
dd, ini, act = chk[:tinit]

act.ActorStationaryPlasma.convergence_error = 2E-2
act.ActorStationaryPlasma.max_iter = 5

act.ActorCoreTransport.model = :FluxMatcher

act.ActorFluxMatcher.step_size = 0.1
act.ActorFluxMatcher.optimizer_algorithm = :simple
act.ActorFluxMatcher.max_iterations = 1000
act.ActorFluxMatcher.verbose = true

FUSE.ActorStationaryPlasma(dd, act; do_plot=false, verbose=true)
chk[:stationary] = (dd, ini, act);

In [ ]:
(dd, ini, act) = chk[:stationary];
FUSE.plot_plasma_overview(dd, dd.global_time; min_power=1E6, aggregate_radiation=true)

In [ ]:
FUSE.ProgressMeter.ijulia_behavior(:clear)

dd, ini, act = chk[:stationary]

act.ActorDynamicPlasma.Nt = 60
act.ActorDynamicPlasma.Δt = 300.0

act.ActorDynamicPlasma.evolve_current = true
act.ActorDynamicPlasma.evolve_equilibrium = true
act.ActorDynamicPlasma.evolve_transport = true
act.ActorDynamicPlasma.evolve_hcd = true
act.ActorDynamicPlasma.evolve_pf_active = true
act.ActorDynamicPlasma.evolve_pedestal = true

act.ActorDynamicPlasma.ip_controller = false

FUSE.ActorDynamicPlasma(dd, act; verbose=true);

chk[:time_dep] = dd, ini, act;

In [ ]:
using Interact

#a = @animate for (k, time0) in collect(enumerate(dd.core_sources.time))[3:end]
#    @show (k,time0)
    @manipulate for k in eachindex(dd.core_sources.time)
    time0 = dd.core_sources.time[k]
    FUSE.plot_plasma_overview(dd, time0; min_power=1E6, aggregate_radiation=true)
    #    savefig(joinpath("/Users/meneghini/Library/CloudStorage/Dropbox/tex/figures/FUSE/gif", "frame_$(lpad(k-1, 4, '0')).png"));
end

In [ ]:
#g = gif(a, "ITER_time_dep_v5.gif", fps=12)
g = gif(a; fps=12)
display(g)